In [1]:
import torch

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, Subset
from PIL import Image, UnidentifiedImageError
import os
import numpy as np

In [3]:
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
import torchvision
train_dataset = torchvision.datasets.GTSRB(root = 'data', split = 'train', download = True, transform = data_transforms)

100.0%


Extracting data\gtsrb\GTSRB-Training_fixed.zip to data\gtsrb


In [5]:
portion = 0.1   # Use 10% of the dataset for training and validation
total_size = len(train_dataset)
subset_size = int(portion * total_size)
indices = np.random.choice(total_size, subset_size, replace=False)

In [6]:
train_size = int(0.8 * subset_size)
val_size = subset_size - train_size
train_indices = indices[:train_size]
val_indices = indices[train_size:]

In [7]:
trainset = DataLoader(torch.utils.data.Subset(train_dataset,train_indices), batch_size =64)
testset = DataLoader(torch.utils.data.Subset(train_dataset,val_indices), batch_size = 64)

In [8]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, padding =1, kernel_size = 3, stride = 1) #64x64
    self.conv2 = nn.Conv2d(16, 32, padding =1, kernel_size = 3, stride = 1) #32x32
    self.conv3 = nn.Conv2d(32, 64, padding =1, kernel_size = 3, stride = 1) #16x16
    self.dropout = nn.Dropout(0.25)
    self.fc1 = nn.Linear(64*16*16, 256)
    self.fc2 = nn.Linear(256,43)
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x,2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x,2)
    x = F.relu(self.conv3(x))
    x = F.max_pool2d(x,2)
    x = self.dropout(x)
    x = torch.flatten(x,1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    return x

In [9]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0005)

In [10]:
epochs = 24
for epoch in range(epochs):
  running_loss = 0.0
  for images, labels in trainset:
    outputs = model(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss +=loss.item()
  print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(trainset)}')

Epoch [1/24], Loss: 3.3197740456637215
Epoch [2/24], Loss: 2.3066812332938698
Epoch [3/24], Loss: 1.6064892831970663
Epoch [4/24], Loss: 1.1507232180413078
Epoch [5/24], Loss: 0.8214704731807989
Epoch [6/24], Loss: 0.6198435837731642
Epoch [7/24], Loss: 0.46641354964059945
Epoch [8/24], Loss: 0.34620540383655357
Epoch [9/24], Loss: 0.22713777510558858
Epoch [10/24], Loss: 0.2016694699140156
Epoch [11/24], Loss: 0.14924996202085716
Epoch [12/24], Loss: 0.09905953023253995
Epoch [13/24], Loss: 0.09541747942292954
Epoch [14/24], Loss: 0.07777083522694953
Epoch [15/24], Loss: 0.08402570917764131
Epoch [16/24], Loss: 0.06329872851561316
Epoch [17/24], Loss: 0.07975270890849917
Epoch [18/24], Loss: 0.053731728338307756
Epoch [19/24], Loss: 0.04493146948516369
Epoch [20/24], Loss: 0.08741359303445172
Epoch [21/24], Loss: 0.03969023177556023
Epoch [22/24], Loss: 0.029252265583422473
Epoch [23/24], Loss: 0.016126575289116075
Epoch [24/24], Loss: 0.0226264392669477


In [11]:
correct = 0
total = 0
preds = []
label = []
with torch.no_grad():
    for images, labels in testset:
        label += labels.tolist()
        preds += list(model(images).argmax(dim=1).numpy())
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the model on the test images: {:.2f}%'.format(100 * correct / total))

Accuracy of the model on the test images: 86.68%


In [29]:
data = {'data_transforms' : data_transforms}
import pickle
with open('saved_steps.pkl','wb') as file:
    pickle.dump(data,file)

In [30]:
with open('saved_steps.pkl','rb') as file:
    data = pickle.load(file)

In [41]:
with open('model.pt', 'wb') as file:
    torch.save(model, file)

In [44]:
with open('model.pt', 'rb') as file:
    the_model = torch.load(file)

C:\Users\SURFACE BOOK\AppData\Local\Temp\ipykernel_13652\3874137040.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  the_model = torch.load(file)


In [49]:
the_model

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=16384, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=43, bias=True)
)

In [50]:
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('model_scripted.pt')

In [51]:
model = torch.jit.load('model_scripted.pt')
model.eval()

RecursiveScriptModule(
  original_name=Net
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (conv2): RecursiveScriptModule(original_name=Conv2d)
  (conv3): RecursiveScriptModule(original_name=Conv2d)
  (dropout): RecursiveScriptModule(original_name=Dropout)
  (fc1): RecursiveScriptModule(original_name=Linear)
  (fc2): RecursiveScriptModule(original_name=Linear)
)